In [1]:
pip install transformers

     |████████████████████████████████| 1.0MB 5.2MB/s 
     |████████████████████████████████| 1.1MB 28.5MB/s 
     |████████████████████████████████| 3.0MB 39.7MB/s 
     |████████████████████████████████| 890kB 37.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=62fb68e89a42582f61f1be14f7342b112fbbb82b44a7cec0e4b73a92c96e8b64
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [1]:
pip install tokenizers

     |████████████████████████████████| 3.0MB 4.8MB/s 


In [5]:
import requests

url = 'https://oscar-public.huma-num.fr/shuffled/is.txt.gz'

target_path = 'is.txt.gz'

response = requests.get(url, stream=True)
if response.status_code == 200:
  with open(target_path, 'wb') as f:
    f.write(response.raw.read())

In [8]:
import gzip

with gzip.open('is.txt.gz', 'rb') as f:
  with open('is.txt', 'wb') as a:
    a.write(f.read())

In [ ]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("*.txt")]
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# Save files to disk
tokenizer.save_model("./icelandic")

In [ ]:
import tokenizers
from transformers import RobertaConfig
from transformers import RobertaTokenizerFast
from transformers import RobertaForMaskedLM
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

tokenizers = RobertaTokenizerFast.from_pretrained("./icelandic", max_len=512)
model = RobertaForMaskedLM(config=config)

print(model.num_parameters())

dataset = LineByLineTextDataset(
    tokenizer=tokenizers,
    file_path="./is.txt",
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizers, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    output_dir="./icelandic",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)


trainer.train()
trainer.save_model("./icelandic")
